<a href="https://colab.research.google.com/github/LCaravaggio/felicidad_ivqreg/blob/main/GeoCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://ookla-open-data.s3.amazonaws.com/shapefiles/performance/type=fixed/year=2020/quarter=2/2020-04-01_performance_fixed_tiles.zip"

--2023-10-06 14:22:20--  https://ookla-open-data.s3.amazonaws.com/shapefiles/performance/type=fixed/year=2020/quarter=2/2020-04-01_performance_fixed_tiles.zip
Resolving ookla-open-data.s3.amazonaws.com (ookla-open-data.s3.amazonaws.com)... 52.92.248.41, 52.92.147.113, 52.218.132.67, ...
Connecting to ookla-open-data.s3.amazonaws.com (ookla-open-data.s3.amazonaws.com)|52.92.248.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 428834336 (409M) [application/zip]
Saving to: ‘2020-04-01_performance_fixed_tiles.zip’

2020-04-01_performa 100%[===================>] 408.97M  12.2MB/s    in 33s     

2023-10-06 14:22:54 (12.3 MB/s) - ‘2020-04-01_performance_fixed_tiles.zip’ saved [428834336/428834336]



In [ ]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [ ]:
!pip install simpledbf
from simpledbf import Dbf5
dbf = Dbf5('/content/gps_fixed_tiles.dbf')
df = dbf.to_dataframe()

In [ ]:
!pip install pyquadkey2

from pyquadkey2 import quadkey

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
def qk_to_geo(quadkey_str, zoom_level):
    qk = quadkey.from_str(quadkey_str)
    lat_lon = qk.to_geo(zoom_level)
    return lat_lon

In [ ]:
df['geo'] = df['quadkey'].apply(qk_to_geo , zoom_level=1)

In [ ]:
import requests
from bs4 import BeautifulSoup

def Citytolatlon(city):
  api_url = 'http://www.geonames.org/search.html?q={}'
  response = requests.get(api_url + city)
  text=""
  if response.status_code == requests.codes.ok:
    soup = BeautifulSoup(response.text, 'html.parser')
    td_elements = soup.find_all('td', nowrap=True)
    count = 0
    for td in td_elements:
      text += td.get_text(strip=True)
      count+=1

      if count == 2:
          break
  else:
      print("Error:", response.status_code, response.text)
  return text

In [ ]:
!pip install latlon3
import latlon

In [ ]:
from latlon import Latitude, Longitude, string2latlon

In [ ]:
import re

class LL:
    __name__ = 'LatLon'
    def __init__(self, lat, lon, name=None):

        try:
            if lat.type() == 'GeoCoord':
                self.lat = lat
            else:
                raise AttributeError
        except AttributeError:
            self.lat = Latitude(lat)
        try:
            if lon.type() == 'GeoCoord':
                self.lon = lon
            else:
                raise AttributeError
        except AttributeError:
            self.lon = Longitude(lon)
        self.name = name

def toLatLong (texto):
  pattern = r'([SN])\s+(\d+)°\s+(\d+)\'\s+(\d+)\'\'([EW])\s+(\d+)°\s+(\d+)\'\s+(\d+)\'\''
  matches = re.findall(pattern, texto.replace('�','°'))

  if matches:
    direction1, degrees1, minutes1, seconds1, direction2, degrees2, minutes2, seconds2 = matches[0]
    output_str1 = f"{degrees1} {minutes1} {seconds1} {direction1}"
    output_str2 = f"{degrees2} {minutes2} {seconds2} {direction2}"

  else:
    return LL(lat=0, lon=0)

  return string2latlon(output_str1, output_str2, 'd% %m% %S% %H')

In [ ]:
import pandas as pd
base=pd.read_stata('LB2020.dta')

In [ ]:
lista_ciudades=base['ciudad'].unique()

In [ ]:
latsylongs=[]
for ciudad in lista_ciudades:
    # Get the latitude and longitude for the current city
    lat_lon = toLatLong(Citytolatlon(ciudad.encode('latin-1').decode('utf-8')))

    # Append the lat_lon values as a list to the matrix
    latsylongs.append([ciudad, lat_lon.lat, lat_lon.lon])

# Create a DataFrame from the list of lists
ciudades_latlon = pd.DataFrame(latsylongs, columns=['City', 'Latitude', 'Longitude'])

In [ ]:
ciudades_latlon

,City,Latitude,Longitude
0,AR: Buenos Aires-Presidente Peron,-34.91777777777777,-58.37916666666667
1,AR: Capital Federal-Ciudad de Buenos Aires,-34.613055555555555,-58.37722222222222
2,AR: Buenos Aires-Suipacha,-34.75,-59.75
3,AR: Buenos Aires-Necochea,-38.55444444444444,-58.739444444444445
4,AR: TucumÃ¡n-Lules,-26.928611111111113,-65.33833333333332
...,...,...,...
1367,VE: Miranda-Acevedo,10.25,-66.33333333333333
1368,VE: Monagas-Caripe,10.166666666666666,-63.416666666666664
1369,VE: Trujillo-Candelaria,9.618055555555555,-70.35694444444444
1370,VE: Trujillo-Carache,9.666666666666666,-70.33333333333333


In [ ]:
ciudades_latlon['Latitude_num']=ciudades_latlon['Latitude'].astype(float)
ciudades_latlon['Longitude_num']=ciudades_latlon['Longitude'].astype(float)

In [ ]:
(ciudades_latlon['Latitude_num'] != 0).sum()

1166

In [ ]:
from scipy.spatial import cKDTree
import numpy as np

kdtree = cKDTree(df['geo'].apply(lambda x: (np.radians(x[0]), np.radians(x[1]))).tolist())

def find_nearest_city(row):
    lat, lon = np.radians(row['Latitude_num']), np.radians(row['Longitude_num'])
    dist, idx = kdtree.query([lat, lon])
    return df.at[idx, 'avg_d_kbps']

ciudades_latlon['avg_d_kbps'] = ciudades_latlon.apply(find_nearest_city, axis=1)

In [ ]:
ciudades_latlon

,City,Latitude,Longitude,Latitude_num,Longitude_num,avg_d_kbps
0,AR: Buenos Aires-Presidente Peron,-34.91777777777777,-58.37916666666667,-34.917778,-58.379167,174179
1,AR: Capital Federal-Ciudad de Buenos Aires,-34.613055555555555,-58.37722222222222,-34.613056,-58.377222,174489
2,AR: Buenos Aires-Suipacha,-34.75,-59.75,-34.750000,-59.750000,11339
3,AR: Buenos Aires-Necochea,-38.55444444444444,-58.739444444444445,-38.554444,-58.739444,103360
4,AR: TucumÃ¡n-Lules,-26.928611111111113,-65.33833333333332,-26.928611,-65.338333,91275
...,...,...,...,...,...,...
1367,VE: Miranda-Acevedo,10.25,-66.33333333333333,10.250000,-66.333333,4933
1368,VE: Monagas-Caripe,10.166666666666666,-63.416666666666664,10.166667,-63.416667,4025
1369,VE: Trujillo-Candelaria,9.618055555555555,-70.35694444444444,9.618056,-70.356944,2675
1370,VE: Trujillo-Carache,9.666666666666666,-70.33333333333333,9.666667,-70.333333,2403


In [ ]:
ciudades_latlon.to_csv('ciudades_latlon.csv')